In [11]:
from tqdm import tqdm
from datetime import datetime

import requests
import json
import time
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Cookie": "_xsrf=HkbVjsizCnXPjOFVMN3P4q1iKzLwFHwT; _zap=fba3ec69-894a-4061-b86a-a1bb6af2141c; d_c0=AEASw0KszRmPTjjL8TCwKLF1t8BnT9kgQ1Y=|1736079203; z_c0=2|1:0|10:1764436703|4:z_c0|92:Mi4xMnNYelFBQUFBQUFBUUJMRFFxek5HU1lBQUFCZ0FsVk4zM2dZYWdCLWpUeWxodjF0eUhEYUNTcUlZaEpXa0xYdEVn|3f9a4db331f712d7f27a09f19cf8a3512e14b55f00f8a57db26d4ca0b7f5d737; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1764436400,1764480124,1764755315,1765221512; __zse_ck=004_djqLe54GMji5CVVrgc4q9LV8IAPjofjtFiOUmKbDZeElq7HVVto/Q3wgf7dbmseo3Y3ZCLmH9//5Dn=kLZaso3lUaxkeXPG0otIiPuA9kb8WyAUDiYW86qP9mx8fvoYj-rVF9qdQWrv11WG9wcX/7OZrbJPvkFclUjF2Mi7N7IXAHhwM5V66gsCguw74pY22eoAnKepbzDi42blWXa4G6gzJkm0hUiz0P6PnhKe7cKVkc/k1W3Y1QwlHB7C043VWi; BEC=46faae78ffea44ab7c29d705bdab5c18",
    "x-requested-with": "XMLHttpRequest"
}

def search_question_ids(keyword, max_pages=20):
    question_ids = []

    for page in tqdm(range(max_pages)):
        url = "https://www.zhihu.com/api/v4/search_v3"
        params = {
            "q": keyword,
            "t": "general",
            "offset": page * 20,
            "limit": 20
        }

        resp = requests.get(url, headers=headers, params=params, timeout=10)
        if resp.status_code != 200:
            break

        data = resp.json()
        for item in data.get("data", []):
            obj = item.get("object", {})
            if obj.get("type") == "question":
                question_ids.append(obj.get("id"))

        time.sleep(1)

    return question_ids

def get_answers(question_id, limit=20, max_answers=30):
    offset = 0
    answers = []

    while True:
        url = f"https://www.zhihu.com/api/v4/questions/{question_id}/answers"
        params = {"limit": limit, "offset": offset, "include": "content,created_time,id"}

        resp = requests.get(url, headers=headers, params=params, timeout=10)
        if resp.status_code != 200:
            break

        data = resp.json()
        for i in data.get("data", []):
            answers.append({
                "source": "zhihu",
                "question_id": question_id,
                "answer_id": i.get("id"),
                "answer": i.get("content", ""),
                "url": f"https://www.zhihu.com/question/{question_id}/answer/{i.get('id')}",
                "timestamp": i.get("created_time")
            })

        if data.get("paging", {}).get("is_end", True):
            break
        if len(answers) >= max_answers:
            break
        offset += limit
        time.sleep(1)

    if len(answers) > max_answers:
        answers = random.sample(answers, max_answers)

    return answers

In [ ]:
AI_KEYWORDS = [
    "人工智能",
    "AI",
    "大模型",
    "ChatGPT",
    "GPT",
    "深度学习",
    "机器学习",
    "算法",
]
question_ids = []
for kw in AI_KEYWORDS:
    ids = search_question_ids(kw, max_pages=50)
    question_ids.extend(ids)
question_ids = list(set(question_ids))  # 去重
print("搜索得到的 question_id:", question_ids)

100%|██████████| 30/30 [00:46<00:00,  1.53s/it]

搜索得到的 question_id: ['1981799426579273408', '1982676571375560531', '1984677760979993684', '1983856245950674576', '1983718834717337506', '1984627506842719996', '1984233504284906318', '1984686099000485817', '1982742854880219839', '1984488576780637690', '1984657424087532972', '1983931217243022467', '1980520428700189609', '1981485486787818628', '1982413280875151595', '1984628616739455537', '1983855940559209968', '576948380', '1984581668573573706', '1984677312994759075', '1984611387578790012', '1984669380219515732', '1984699124185068005', '1984688491091752607', '1984284670809113335']


In [14]:
len(question_ids)

25

In [ ]:
topic = "人工智能"
question_ids = search_question_ids(
    keyword=topic,
    max_pages=30,
    sample_size=50
)

for qid in tqdm(question_ids):
    answers = get_answers(qid)
    with open("zhihu_answers.jsonl", "a", encoding="utf-8") as f:
        for entry in answers:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
    print(f"[OK] question {qid} -> {len(answers)} answers")

In [ ]:
url = "https://www.zhihu.com/api/v4/search_v3"
params = {
    "q": "人工智能",
    "t": "question",
    "offset": 0,
    "limit": 1
}

data = requests.get(url, headers=headers, params=params, timeout=10).json()

for item in data.get('data', []):
    if item.get('type') == 'search_result':
        obj = item.get('object', {})
        if obj.get('type') == 'question':
            question_ids.append(obj.get('id'))
    
question_ids

In [ ]:
import requests
import json
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Cookie": "_xsrf=HkbVjsizCnXPjOFVMN3P4q1iKzLwFHwT; _zap=fba3ec69-894a-4061-b86a-a1bb6af2141c; d_c0=AEASw0KszRmPTjjL8TCwKLF1t8BnT9kgQ1Y=|1736079203; z_c0=2|1:0|10:1764436703|4:z_c0|92:Mi4xMnNYelFBQUFBQUFBUUJMRFFxek5HU1lBQUFCZ0FsVk4zM2dZYWdCLWpUeWxodjF0eUhEYUNTcUlZaEpXa0xYdEVn|3f9a4db331f712d7f27a09f19cf8a3512e14b55f00f8a57db26d4ca0b7f5d737; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1764436400,1764480124,1764755315,1765221512; __zse_ck=004_djqLe54GMji5CVVrgc4q9LV8IAPjofjtFiOUmKbDZeElq7HVVto/Q3wgf7dbmseo3Y3ZCLmH9//5Dn=kLZaso3lUaxkeXPG0otIiPuA9kb8WyAUDiYW86qP9mx8fvoYj-rVF9qdQWrv11WG9wcX/7OZrbJPvkFclUjF2Mi7N7IXAHhwM5V66gsCguw74pY22eoAnKepbzDi42blWXa4G6gzJkm0hUiz0P6PnhKe7cKVkc/k1W3Y1QwlHB7C043VWi; BEC=46faae78ffea44ab7c29d705bdab5c18",
    "x-requested-with": "XMLHttpRequest"
}

def fetch_answer(question_id, limit=20):
    offset = 0
    answers = []

    while True:
        url = f"https://www.zhihu.com/api/v4/questions/{question_id}/answers"
        # url = f"https://www.zhihu.com/question/{question_id}"
        params = {"limit": limit, "offset": offset, "include": "content,created_time,id"}
        resp = requests.get(url, headers=headers, params=params, timeout=10)
        if resp.status_code != 200:
            print(f"请求失败: {resp.status_code}")
            break

        data = resp.json()
        for i in data.get("data", []):
            answers.append({
                "source": "zhihu",
                "question_id": question_id,
                "answer_id": i.get("id"),
                "answer": i.get("content", ""),
                "url": f"https://www.zhihu.com/question/{question_id}/answer/{i.get('id')}",
                "timestamp": i.get("created_time")
            })

        if not data.get("paging", {}).get("is_end", True):
            offset += limit
            time.sleep(1)
        else:
            break

    return answers


def save_jsonl(filename, data):
    with open(filename, "a", encoding="utf-8") as f:
        for entry in data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")


if __name__ == "__main__":
    question_ids = ["123456789"]
    for qid in question_ids:
        answers = fetch_answer(qid)
        save_jsonl("zhihu_answers.jsonl", answers)
        print(f"已保存 {len(answers)} 条回答")
